In [245]:
import xlrd
data = xlrd.open_workbook(r'/home/liang.liang/code/jupyterCode/workitems.xlsx')
table = data.sheets()[0]


keyId=table.col_values(2,1)
delayHours=table.col_values(3,1)
functionGroup=table.col_values(4,1)
status=table.col_values(5,1)

feature_Counts = len(keyId)
status_Counts = 5

x_keyid_group = []
x_keyid_status = []

y_delayDays = []
count = 0



In [246]:
keyId

['1.11.1', '1.11.3', '1.11.6', '1.5', '1.7', '1.8', '1.9.2']

In [247]:
delayHours

[50.2, 50.2, 162.2, 162.2, 50.2, 162.2, 162.2]

In [248]:
functionGroup

['Radio', 'Radio', 'Radio', 'Radio', 'Radio', 'Radio', 'Radio']

In [249]:
status

['New', 'New', 'New', 'New', 'New', 'New', 'New']

In [250]:
feature_Counts

7

In [251]:
count = 0
while(count < feature_Counts ):
    x3 = keyId[count] + '['+functionGroup[count] + ']'        
    x_keyid_group.append(x3)
    count  = count + 1

In [252]:
x_keyid_group

['1.11.1[Radio]',
 '1.11.3[Radio]',
 '1.11.6[Radio]',
 '1.5[Radio]',
 '1.7[Radio]',
 '1.8[Radio]',
 '1.9.2[Radio]']

In [253]:
count = 0
while(count < feature_Counts ):
    x3 = keyId[count] + '['+status[count] + ']'        
    x_keyid_status.append(x3)
    count  = count + 1

In [254]:
x_keyid_status

['1.11.1[New]',
 '1.11.3[New]',
 '1.11.6[New]',
 '1.5[New]',
 '1.7[New]',
 '1.8[New]',
 '1.9.2[New]']

In [255]:
for x in delayHours:
    y_delayDays.append(int(x)/8)

In [256]:
y_delayDays

[6.25, 6.25, 20.25, 20.25, 6.25, 20.25, 20.25]

In [257]:
count = 0
status_dict = {"New":[],"In Analysis":[],"Pending":[],"In Implementation":[],"In Testing":[]}

status_list = ["New","In Analysis","Pending","In Implementation","In Testing"]

for x in status_list:

    for y in range(feature_Counts):
        
        status_dict[x].append(0)

for st in range(feature_Counts):
    status_dict[status[st]][st] = y_delayDays[st]

In [258]:
status_dict

{'New': [6.25, 6.25, 20.25, 20.25, 6.25, 20.25, 20.25],
 'In Analysis': [0, 0, 0, 0, 0, 0, 0],
 'Pending': [0, 0, 0, 0, 0, 0, 0],
 'In Implementation': [0, 0, 0, 0, 0, 0, 0],
 'In Testing': [0, 0, 0, 0, 0, 0, 0]}

In [259]:
count = 0
funcGroup_dict = {"Android":[],"AppCon":[],"Audio":[],"BT":[],"BT_Org":[],"HMI":[],
                    "HU":[],"Kombi":[],"MAP":[],"Media":[],"MFL":[],"Navi":[],
                    "Online":[],"ORU":[],"PSO2.0":[],"Radio":[],"SDS":[],"SPApp":[],
                    "Software":[],"Speech":[],"SWDL":[],"System":[],"WiFi":[],"Diagnose":[]
                   }

funcGroup_list = ["Android","AppCon","Audio","BT","BT_Org","HMI",
                    "HU","Kombi","MAP","Media","MFL","Navi",
                    "Online","ORU","PSO2.0","Radio","SDS","SPApp",
                    "Software","Speech","SWDL","System","WiFi","Diagnose"
                 ]



for x in funcGroup_list:

    for y in range(feature_Counts):
        
        funcGroup_dict[x].append(0)

for st in range(feature_Counts):
    funcGroup_dict[functionGroup[st]][st] = y_delayDays[st]

In [260]:
funcGroup_dict

{'Android': [0, 0, 0, 0, 0, 0, 0],
 'AppCon': [0, 0, 0, 0, 0, 0, 0],
 'Audio': [0, 0, 0, 0, 0, 0, 0],
 'BT': [0, 0, 0, 0, 0, 0, 0],
 'BT_Org': [0, 0, 0, 0, 0, 0, 0],
 'HMI': [0, 0, 0, 0, 0, 0, 0],
 'HU': [0, 0, 0, 0, 0, 0, 0],
 'Kombi': [0, 0, 0, 0, 0, 0, 0],
 'MAP': [0, 0, 0, 0, 0, 0, 0],
 'Media': [0, 0, 0, 0, 0, 0, 0],
 'MFL': [0, 0, 0, 0, 0, 0, 0],
 'Navi': [0, 0, 0, 0, 0, 0, 0],
 'Online': [0, 0, 0, 0, 0, 0, 0],
 'ORU': [0, 0, 0, 0, 0, 0, 0],
 'PSO2.0': [0, 0, 0, 0, 0, 0, 0],
 'Radio': [6.25, 6.25, 20.25, 20.25, 6.25, 20.25, 20.25],
 'SDS': [0, 0, 0, 0, 0, 0, 0],
 'SPApp': [0, 0, 0, 0, 0, 0, 0],
 'Software': [0, 0, 0, 0, 0, 0, 0],
 'Speech': [0, 0, 0, 0, 0, 0, 0],
 'SWDL': [0, 0, 0, 0, 0, 0, 0],
 'System': [0, 0, 0, 0, 0, 0, 0],
 'WiFi': [0, 0, 0, 0, 0, 0, 0],
 'Diagnose': [0, 0, 0, 0, 0, 0, 0]}

In [261]:
from pyecharts import Bar


bar = Bar("37W Project", "Delay Days for Features")
bar.add("New", x_keyid_group,status_dict["New"], xaxis_rotate=30,xaxis_interval= 0,mark_line=["average"]
       ,is_label_show=False,label_formatter='{c}',is_more_utils=True
       ,xaxis_type = 'category',yaxis_formatter = 'days',is_stack = True,label_pos = 'top')
bar.add("In Analysis", x_keyid_group, status_dict["In Analysis"], xaxis_rotate=30,xaxis_interval= 0,mark_line=["average"]
       ,is_label_show=False,label_formatter='{c}',is_more_utils=True
       ,xaxis_type = 'category',yaxis_formatter = 'days',is_stack = True,label_pos = 'top')




In [262]:
from pyecharts import Bar


bar = Bar("37W Project", "Delay Days for Features")
bar.add("Android", x_keyid_status,funcGroup_dict["Android"], xaxis_rotate=30,xaxis_interval= 0,mark_line=["average"]
       ,is_label_show=False,label_formatter='{c}',is_more_utils=True
       ,xaxis_type = 'category',yaxis_formatter = 'days',is_stack = True,label_pos = 'top')
bar.add("AppCon", x_keyid_status, funcGroup_dict["AppCon"], xaxis_rotate=30,xaxis_interval= 0,mark_line=["average"]
       ,is_label_show=False,label_formatter='{c}',is_more_utils=True
       ,xaxis_type = 'category',yaxis_formatter = 'days',is_stack = True,label_pos = 'top')

bar.add("Radio", x_keyid_status, funcGroup_dict["Radio"], xaxis_rotate=30,xaxis_interval= 0,mark_line=["average"]
       ,is_label_show=False,label_formatter='{c}',is_more_utils=True
       ,xaxis_type = 'category',yaxis_formatter = 'days',is_stack = True,label_pos = 'top')

In [274]:
# funcGroup_list

#{"New":0,"In Analysis":1,"Pending":2,"In Implementation":3,"In Testing":4}


funcGroup_total ={"New":
                  {"Android":0,"AppCon":0,"Audio":0,"BT":0,"BT_Org":0,"HMI":0,
                    "HU":0,"Kombi":0,"MAP":0,"Media":0,"MFL":0,"Navi":0,
                    "Online":0,"ORU":0,"PSO2.0":0,"Radio":0,"SDS":0,"SPApp":0,
                    "Software":0,"Speech":0,"SWDL":0,"System":0,"WiFi":0,"Diagnose":0
                   },
                 "In Analysis":
                  {"Android":0,"AppCon":0,"Audio":0,"BT":0,"BT_Org":0,"HMI":0,
                    "HU":0,"Kombi":0,"MAP":0,"Media":0,"MFL":0,"Navi":0,
                    "Online":0,"ORU":0,"PSO2.0":0,"Radio":0,"SDS":0,"SPApp":0,
                    "Software":0,"Speech":0,"SWDL":0,"System":0,"WiFi":0,"Diagnose":0
                   },
                  "Pending":
                  {"Android":0,"AppCon":0,"Audio":0,"BT":0,"BT_Org":0,"HMI":0,
                    "HU":0,"Kombi":0,"MAP":0,"Media":0,"MFL":0,"Navi":0,
                    "Online":0,"ORU":0,"PSO2.0":0,"Radio":0,"SDS":0,"SPApp":0,
                    "Software":0,"Speech":0,"SWDL":0,"System":0,"WiFi":0,"Diagnose":0
                   },
                  "In Implementation":
                   {"Android":0,"AppCon":0,"Audio":0,"BT":0,"BT_Org":0,"HMI":0,
                    "HU":0,"Kombi":0,"MAP":0,"Media":0,"MFL":0,"Navi":0,
                    "Online":0,"ORU":0,"PSO2.0":0,"Radio":0,"SDS":0,"SPApp":0,
                    "Software":0,"Speech":0,"SWDL":0,"System":0,"WiFi":0,"Diagnose":0
                   },
                  "In Testing":
                  {"Android":0,"AppCon":0,"Audio":0,"BT":0,"BT_Org":0,"HMI":0,
                    "HU":0,"Kombi":0,"MAP":0,"Media":0,"MFL":0,"Navi":0,
                    "Online":0,"ORU":0,"PSO2.0":0,"Radio":0,"SDS":0,"SPApp":0,
                    "Software":0,"Speech":0,"SWDL":0,"System":0,"WiFi":0,"Diagnose":0
                  }
                  
                 }

for i in range(feature_Counts):
     funcGroup_total[status[i]][functionGroup[i]] += y_delayDays[i]


status_total = {"New":[],"In Analysis":[],"Pending":[],"In Implementation":[],"In Testing":[]}

    